# How to prepare test_data when train data has Lagged Columns
> When the training data has lagged columns as the features, it becomes hard to prepare test_dataset for future prediction, <br>
> Preparing test mean not 20% reserved data of train_test set, but if 100% of data goes in train data, and test as in predicting tomorrow or next week, i.e. future data not available <br>
> In such cases, knowing the lagged values of day after tomorrow is not possible as we don't have data for tomorrow <br>
> But having it is a necessity, as it's a part of training dataset 
 <br>
 
> These cases can be solved by <br>
>  if instead of training the model at day level, we train them at week level <br>
> and then predict 1 time stamp ahead, i.e. 1 week ahead

![](./right_shift_columns_01.png)
![](./right_shift_columns_02.png)

In [1]:
import pandas as pd
import numpy as np
import random

# Creating Base Data: Week Level Data

In [2]:
# Generate a list of ATM IDs
atm_ids = ['ATM1', 'ATM2', 'ATM3']

# Generate a date range from 1st January 2023 to 20th February 2023
date_range = pd.date_range(start='2023-01-05', end='2023-02-20')

# Create an empty DataFrame
base_df = pd.DataFrame()

# Populate the DataFrame for each ATM ID
for atm_id in atm_ids:
    temp_df = pd.DataFrame({
        'ATM_ID': [atm_id]*len(date_range),
        'Date': date_range,
        'Holiday': [random.choice([0, 1]) for _ in range(len(date_range))],
        'Withdrawal_Amount': np.random.randint(1000, 10000, size=len(date_range))  # Random withdrawal amounts        
    })
    base_df = pd.concat([base_df, temp_df], ignore_index=True)
base_df['Date'] = pd.to_datetime(base_df['Date'])

# Update code to use the recommended method for generating week number
base_df['Week_Number'] = base_df['Date'].dt.isocalendar().week
base_df['Year'] = base_df['Date'].dt.isocalendar().year

# Display first few records to show the added Week_Number and Year columns
base_df.head()

,ATM_ID,Date,Holiday,Withdrawal_Amount,Week_Number,Year
0,ATM1,2023-01-05,0,6980,1,2023
1,ATM1,2023-01-06,0,5104,1,2023
2,ATM1,2023-01-07,0,7120,1,2023
3,ATM1,2023-01-08,1,9601,1,2023
4,ATM1,2023-01-09,1,6067,2,2023


# Week level data aggregation

In [3]:
# Now, let's proceed with the groupby and aggregation
grouped_df = base_df.groupby(['ATM_ID', 'Year', 'Week_Number']).agg({
    'Withdrawal_Amount': 'sum',
    'Holiday': lambda x: 1 if any(x) else 0  # Flag as 1 if any record in the group has Holiday=1, else 0
}).reset_index()
grouped_df = grouped_df[[
    'ATM_ID', 'Year', 'Holiday', 'Week_Number', 'Withdrawal_Amount'
       ]]
grouped_df.head()

,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount
0,ATM1,2023,1,1,28805
1,ATM1,2023,1,2,41622
2,ATM1,2023,1,3,31857
3,ATM1,2023,1,4,30657
4,ATM1,2023,1,5,28929


In [4]:
grouped_df.loc[grouped_df['Holiday']==0]#.sample(2)

,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount
7,ATM1,2023,0,8,4854
15,ATM2,2023,0,8,5323
23,ATM3,2023,0,8,2289


# Creating 5-Week lag columns

In [5]:
# Creating lag features for 'Withdrawal_Amount' ranging from 1 to 5
lagged_df = grouped_df.copy()
for lag in range(1, 6):
    lagged_df[f'Withdrawal_Amount_Lag_{lag}'] = lagged_df.groupby(['ATM_ID'])['Withdrawal_Amount'].shift(lag)

# Display the first few records to show the added lag features
lagged_df.head(12)

,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount,Withdrawal_Amount_Lag_1,Withdrawal_Amount_Lag_2,Withdrawal_Amount_Lag_3,Withdrawal_Amount_Lag_4,Withdrawal_Amount_Lag_5
0,ATM1,2023,1,1,28805,NaN,NaN,NaN,NaN,NaN
1,ATM1,2023,1,2,41622,28805.0,NaN,NaN,NaN,NaN
2,ATM1,2023,1,3,31857,41622.0,28805.0,NaN,NaN,NaN
3,ATM1,2023,1,4,30657,31857.0,41622.0,28805.0,NaN,NaN
4,ATM1,2023,1,5,28929,30657.0,31857.0,41622.0,28805.0,NaN
5,ATM1,2023,1,6,43411,28929.0,30657.0,31857.0,41622.0,28805.0
6,ATM1,2023,1,7,40642,43411.0,28929.0,30657.0,31857.0,41622.0
7,ATM1,2023,0,8,4854,40642.0,43411.0,28929.0,30657.0,31857.0
8,ATM2,2023,1,1,14904,NaN,NaN,NaN,NaN,NaN
9,ATM2,2023,1,2,28872,14904.0,NaN,NaN,NaN,NaN


# Creating `test_df` for prediction in prod_script

In [6]:
# Displaying the last row for each combination of 'ATM_ID' and 'Year'
last_row_per_group = lagged_df.groupby(['ATM_ID', 'Year']).last().reset_index()
last_row_per_group

,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount,Withdrawal_Amount_Lag_1,Withdrawal_Amount_Lag_2,Withdrawal_Amount_Lag_3,Withdrawal_Amount_Lag_4,Withdrawal_Amount_Lag_5
0,ATM1,2023,0,8,4854,40642.0,43411.0,28929.0,30657.0,31857.0
1,ATM2,2023,0,8,5323,40272.0,38175.0,35195.0,35354.0,37343.0
2,ATM3,2023,0,8,2289,34965.0,33093.0,36613.0,34694.0,29531.0


In [7]:
test_df = last_row_per_group.copy()
test_df['Week_Number'] = test_df['Week_Number']+1

# Columns to be shifted
cols_to_shift = ['Withdrawal_Amount', 'Withdrawal_Amount_Lag_1', 'Withdrawal_Amount_Lag_2', 'Withdrawal_Amount_Lag_3', 'Withdrawal_Amount_Lag_4', 'Withdrawal_Amount_Lag_5']

# Create a DataFrame with the first column set to None for the specific columns we want to shift
first_col_for_shift = pd.DataFrame({cols_to_shift[0]: [None]*len(test_df)})

# Use slicing and iloc to select and shift the columns
grouped_df_shifted_partial = pd.concat([first_col_for_shift, test_df[cols_to_shift].iloc[:, :-1]], axis=1)

# Rename the columns to match the original subset of DataFrame's columns
grouped_df_shifted_partial.columns = cols_to_shift

# Replace only the shifted columns in the original DataFrame
test_df[cols_to_shift] = grouped_df_shifted_partial
test_df.drop(columns=['Withdrawal_Amount'], inplace=True)

In [8]:
print("\n\nFor each ATM_ID: Last row of training set")
display(last_row_per_group)

print("\n\nFor each ATM_ID: Input for Test set")
display(test_df)



For each ATM_ID: Last row of training set


,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount,Withdrawal_Amount_Lag_1,Withdrawal_Amount_Lag_2,Withdrawal_Amount_Lag_3,Withdrawal_Amount_Lag_4,Withdrawal_Amount_Lag_5
0,ATM1,2023,0,8,4854,40642.0,43411.0,28929.0,30657.0,31857.0
1,ATM2,2023,0,8,5323,40272.0,38175.0,35195.0,35354.0,37343.0
2,ATM3,2023,0,8,2289,34965.0,33093.0,36613.0,34694.0,29531.0




For each ATM_ID: Input for Test set


,ATM_ID,Year,Holiday,Week_Number,Withdrawal_Amount_Lag_1,Withdrawal_Amount_Lag_2,Withdrawal_Amount_Lag_3,Withdrawal_Amount_Lag_4,Withdrawal_Amount_Lag_5
0,ATM1,2023,0,9,4854,40642.0,43411.0,28929.0,30657.0
1,ATM2,2023,0,9,5323,40272.0,38175.0,35195.0,35354.0
2,ATM3,2023,0,9,2289,34965.0,33093.0,36613.0,34694.0
